In [1]:
%reload_ext autoreload

In [2]:
# Imports

%load_ext autoreload
%autoreload 2

%matplotlib inline
import numpy as np
import random
import csv
import pandas as pd

from fastai.imports import *
from fastai.structured import *
from matplotlib import pyplot
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.metrics import f1_score,\
    accuracy_score, confusion_matrix,\
    precision_score, recall_score,\
    roc_curve, roc_auc_score,\
    cohen_kappa_score, mean_absolute_error,\
    precision_recall_curve, auc,\
    average_precision_score

set_plot_sizes(12,14,16)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
import csv

sys.path.append('../lib')
import utils
import preprocessing

In [6]:
# Load in data

PATH = "../data/"
df_raw = pd.read_csv(f'{PATH}churn.csv', low_memory=False, 
                     parse_dates=['canceldate', 'licence_registration_date', 'golive_date'])

In [9]:
print(len(df_raw[df_raw.churned == 0]), df_raw[df_raw.churned == 0].shape)
df_processed = preprocessing.prepare_data(df_raw)
print(len(df_processed), df_processed.shape)
df_filtered = df_processed[df_processed.licence_registration_Year >2017]
df_data, y_data, nas = proc_df(df_filtered, 'churned')
print(len(df_data), df_data.shape)

2780 (2780, 38)
5889 (5889, 101)
1351 (1351, 104)


/Users/justinhennessy/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
joblib_file = "churn_model.pkl"

# Load from file
churn_model = joblib.load(joblib_file)

predictions = churn_model.predict(df_data)
probability = churn_model.predict_proba(df_data)

now = datetime.datetime.now()
array = []
for i in range(len(df_filtered[df_filtered.churned == 0].username)):
    if predictions[i] == 1:
        array.append([now,df_filtered[df_filtered.churned == 0].username.iloc[i],df_filtered[df_filtered.churned == 0].url.iloc[i],df_filtered[df_filtered.churned == 0].annual_revenue.iloc[i],probability[i][1],predictions[i]])
        

print(len(df_raw[df_raw.churned == 0]), len(array))

2780 222


In [11]:
count = 0

churn_concerns = []

for i in range(len(array)):
    if array[i][4] > .7:
        count += 1
        churn_concerns.append(array[i])

In [12]:
print(count)

66


In [13]:
now = datetime.datetime.now()

sorted_array = sorted(churn_concerns, key=lambda x: x[3], reverse=True)

#for i in range(len(sorted_array)):
for i in range(30):
    print(f"{sorted_array[i][0]},{sorted_array[i][1]},{sorted_array[i][2]},{sorted_array[i][3]},{sorted_array[i][4]},{sorted_array[i][5]}")
    with open('../data/predictions.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows(sorted_array)

writeFile.close()

2019-10-10 07:31:46.940558,N052273,logitechshop.neto.com.au,3161843.85,0.7394143268627161,1
2019-10-10 07:31:46.940558,N055220,www.xdw.com.au,800290.23,0.7415727523669452,1
2019-10-10 07:31:46.940558,N054211,www.funeralproducts.com.au,649310.58,0.7402470584997474,1
2019-10-10 07:31:46.940558,N052113,www.willowspoolshop.com.au,485805.76,0.7364196847657454,1
2019-10-10 07:31:46.940558,N056267,wheel-house-bik.neto.com.au,393561.83,0.7424054840039763,1
2019-10-10 07:31:46.940558,N063379,www.sequelbooks.com,347601.38,0.7051699685577298,1
2019-10-10 07:31:46.940558,N059921,www.directcarparts.com.au,330861.34,0.727379740638174,1
2019-10-10 07:31:46.940558,N055984,www.trailersrus.com.au,295827.07,0.7402470584997474,1
2019-10-10 07:31:46.940558,N053827,www.nordicfusion.com.au,257118.25,0.7445886421282522,1
2019-10-10 07:31:46.940558,N061393,www.theoddspoke.com.au,177926.51,0.7039446464643845,1
2019-10-10 07:31:46.940558,N060326,www.trainerhub.net.au,87983.24,0.7276188214815543,1
2019-10-10 07:3